In [2]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import datetime as dt
from datetime import timedelta

In [3]:
burritos_df = pd.read_csv('burritos_summary_data.csv')
burritos_df

,id,Location,Burrito,Neighborhood,Yelp,Google,Cost,overall,Total_Listings,Average_Price_per_Night
0,1,Nico's Taco Shop,California,Carmel Valley,3.0,2.9,7.19,2.60,156,120
1,2,Porkyland,Deborah's special,Carmel Valley,4.0,4.4,8.25,4.00,156,120
2,3,Taco Stand,California,La Jolla,4.5,4.4,7.49,4.20,847,285
3,4,Rigoberto's Taco Shop La Jolla,California,La Jolla,2.5,4.0,6.40,3.00,847,285
4,5,Goody's,Custom,La Jolla,3.5,4.1,7.25,1.80,847,285
5,6,Don Carlos Taco Shop,California,La Jolla,4.0,4.5,7.75,3.70,847,285
6,7,Lolita's taco shop,2 in 1,Kearny Mesa,4.0,4.4,8.75,4.00,13,65
7,8,Pokirrito,Pokirrito classic,Kearny Mesa,4.0,4.3,11.95,4.20,13,65
8,9,Mikes Taco Club,California,Ocean Beach,4.5,4.9,8.59,4.30,522,177
9,10,La Perla Cocina,Addiction,Pacific Beach,4.5,4.7,7.99,2.50,1052,157


In [4]:
listings_df = pd.read_csv('listing_cleaned.csv')
listings_df

,id,name,neighborhood,latitude,longitude,price
0,5570,Ocean front condo on the sand,Mission Bay,32.78430,-117.25258,2050
1,29967,"Great home, 10 min walk to Beach",Pacific Beach,32.80724,-117.25630,225
2,38245,Point Loma: Den downstairs,Roseville,32.74202,-117.21870,75
3,39516,Art Studio Retreat/Rustic Cabin,North Hills,32.73188,-117.11918,74
4,54001,"La Jolla 2 Bdr Cottage: 1 Qn; 2Twns, Blks 2 Ocn",La Jolla,32.81301,-117.26856,80
...,...,...,...,...,...,...
12399,44399731,Perfect Great Studio for Long Stays!,East Village,32.71013,-117.15619,2999
12400,44399747,Classic Spanish Style Apartment in Heart of SD,Normal Heights,32.75769,-117.12094,78
12401,44402703,University Heights 60's Ranch-Style Canyon Home,West University Heights,32.75797,-117.15452,750
12402,44407375,Hip Metro San Diego Studio,Normal Heights,32.75925,-117.11500,95


In [5]:
summary_df = pd.read_csv('summary_data.csv')
summary_df

,id,Location,Burrito,Neighborhood,Yelp,Google,Cost,overall,Total_Listings,Average_Price_per_Night
0,1,Nico's Taco Shop,California,Carmel Valley,3.0,2.9,7.19,2.60,156,120
1,2,Porkyland,Deborah's special,Carmel Valley,4.0,4.4,8.25,4.00,156,120
2,3,Taco Stand,California,La Jolla,4.5,4.4,7.49,4.20,847,285
3,4,Rigoberto's Taco Shop La Jolla,California,La Jolla,2.5,4.0,6.40,3.00,847,285
4,5,Goody's,Custom,La Jolla,3.5,4.1,7.25,1.80,847,285
5,6,Don Carlos Taco Shop,California,La Jolla,4.0,4.5,7.75,3.70,847,285
6,7,Lolita's taco shop,2 in 1,Kearny Mesa,4.0,4.4,8.75,4.00,13,65
7,8,Pokirrito,Pokirrito classic,Kearny Mesa,4.0,4.3,11.95,4.20,13,65
8,9,Mikes Taco Club,California,Ocean Beach,4.5,4.9,8.59,4.30,522,177
9,10,La Perla Cocina,Addiction,Pacific Beach,4.5,4.7,7.99,2.50,1052,157


In [6]:
connection_string = "sqlite:///burritos_bnb.sqlite"
engine = create_engine(connection_string)

In [7]:
# burritos_df.to_sql(name='burritos', con=engine, if_exists='replace', index=False)

In [8]:
# summary_df.to_sql(name='summary', con=engine, if_exists='replace', index=False)

In [9]:
# Create tables
# burritos table
engine.execute("CREATE table burritos (        id int not NULL,        Location varchar not null,        Burrito varchar not null,        Neighborhood varchar not null,        Yelp numeric not null,        Google numeric not null,        Cost numeric not null,        overall numeric not null,        Total_Listings int not null,        Average_Price_per_Night int not NULL,        primary key (id))")

In [10]:
engine.execute("CREATE table listings (        id int not NULL,        name varchar not null,        neighborhood varchar not null,        latitude numeric not null,        longitude numeric not null,        price numeric not null,        primary key (id))")

In [11]:
engine.execute

<bound method Engine.execute of Engine(sqlite:///burritos_bnb.sqlite)>

In [12]:
# load the listings dataframe as sql table
listings_df.to_sql(name='listings', con=engine, if_exists='append', index=False)

In [13]:
listing_data = engine.execute("select * from listings")
for record in listing_data:
    print(record)

(5570, 'Ocean front condo on the sand', 'Mission Bay', 32.7843, -117.25258000000001, 2050)
(29967, 'Great home, 10 min walk to Beach', 'Pacific Beach', 32.80724, -117.2563, 225)
(38245, 'Point Loma: Den downstairs', 'Roseville', 32.742020000000004, -117.2187, 75)
(39516, 'Art Studio Retreat/Rustic Cabin', 'North Hills', 32.73188, -117.11918, 74)
(54001, 'La Jolla 2 Bdr Cottage: 1 Qn; 2Twns, Blks 2 Ocn', 'La Jolla', 32.81301, -117.26856000000001, 80)
(62274, 'charming, colorful, close to beach', 'Pacific Beach', 32.80734, -117.24243, 68)
(62949, 'Pacific Beach Cozy Private Unit Bikes Included', 'Pacific Beach', 32.80961, -117.23056000000001, 75)
(67441, 'Canyon apartment hideaway', 'College Area', 32.76365, -117.07675, 90)
(75668, 'San Diego Living by JA 2', 'North Hills', 32.7473, -117.14423000000001, 45)
(77785, 'Ideal for Work, Med Students, Grad Students', 'Old Town', 32.7481, -117.19165, 150)
(79300, 'San Diego living by JA 1', 'North Hills', 32.745940000000004, -117.14406000000001

(7868382, 'Delightful Getaway to Studio in Gaslamp District', 'Gaslamp Quarter', 32.715, -117.15982, 144)
(7871918, 'Cozy room with private bathroom', 'Tierrasanta', 32.82949, -117.1115, 40)
(7872614, 'Beautiful San Diego HOME   - WEST  Bedroom', 'Bay Park', 32.77964, -117.2068, 46)
(7876894, 'Private room #2, close to downtown!', 'Mountain View', 32.69565, -117.10088, 35)
(7877393, 'Private room #1, close to downtown!', 'Mountain View', 32.69545, -117.10088, 33)
(7883282, 'Private room #3, close to downtown!', 'Mountain View', 32.69713, -117.10283999999999, 33)
(7888043, 'San Diego Beach Studio 55 seconds To The Sand', 'Mission Bay', 32.767509999999994, -117.25218999999998, 59)
(7888241, '#Staycation , Need More Space Beautiful  4 Bedroom', 'Mission Bay', 32.79347, -117.23275, 389)
(7896290, 'Huge loft living in Gaslamp walk to Convention', 'East Village', 32.714620000000004, -117.15763999999999, 160)
(7896416, 'Bright beach friendly apartment', 'Ocean Beach', 32.74283, -117.249580000

(14372805, ' High-End Beach Townhouse  - Central AC, Garage!', 'Pacific Beach', 32.80591, -117.25546999999999, 485)
(14372808, ' Sunny Ground Floor Private Patio ☀️, Walk to ALL!', 'Pacific Beach', 32.7975, -117.25403999999999, 395)
(14372812, ' UPGRADED | FAMILY Beach home- BBQ, Parking, Ground Floor Patio & Roof Deck', 'Mission Bay', 32.78304, -117.25282, 550)
(14372815, ' Quintessential Beach Cottage | Centrally Located | Walk to ALL- Beach & Bay', 'Mission Bay', 32.78371, -117.25196000000001, 300)
(14372822, '☀️ SINGLE FAMILY BEACH HOME  - Centrally located, walking distance to all!', 'Mission Bay', 32.78605, -117.25294, 450)
(14372823, "☀️ *MODERN* High-End Oceanfront Condo  w/ Hot Tub, Sun Deck, BBQ's, 1 Parking!", 'Mission Bay', 32.77691, -117.25299999999999, 495)
(14372824, ' 2020 REMODEL | GROUND FLOOR | Fresh New Family Vacation Rental!', 'Mission Bay', 32.76548, -117.2517, 350)
(14372828, ' *PRIVATE* SINGLE FAMILY  2-Car Garage, Central AC, 1-House from Ocean!', 'Mission Bay

(19721149, 'SDJade Home Zone', 'Sky Line', 32.70588, -117.05117, 45)
(19721192, 'Private Guest Room in Talmadge/Kensington', 'Talmadge', 32.758140000000004, -117.08226, 140)
(19721306, 'Spectacular San Diego home!  Close to everything!', 'Balboa Park', 32.71367, -117.14572, 900)
(19722002, "Oh, the Places You'll Go...Get Craft Beer! Cottage", 'North Hills', 32.75087, -117.12778999999999, 95)
(19722750, 'Private day room', 'Clairemont Mesa', 32.815709999999996, -117.16828000000001, 82)
(19728637, 'Cozy Condo, Central SD Location!!!', 'Grantville', 32.7827, -117.11073999999999, 90)
(19732921, 'Adorable North Park Cottage with Private Yard', 'North Hills', 32.74852, -117.13396999999999, 140)
(19734051, 'California Dreams – Bed in a Beach Hostel Dorm', 'Pacific Beach', 32.7982, -117.25643000000001, 40)
(19734213, 'Jacaranda View two room Studio', 'Cortez Hill', 32.72155, -117.15683999999999, 136)
(19734757, 'California Dreams – Bed in Fun Beach Hostel', 'Pacific Beach', 32.7982, -117.25643

(24199172, "Jenny's Casita, Centrally Located Spanish Bungalow", 'North Hills', 32.76587, -117.13105, 99)
(24199888, 'New & Modern 3bedroom Apt in heart of Downtown SD', 'Marina', 32.71071, -117.16264, 200)
(24200988, 'Bright modern 2bd, 2ba Townhome - beautiful views!', 'Moreno Mission', 32.766090000000005, -117.18807, 199)
(24201328, 'Sabre springs Wimberly place', 'Sabre Springs', 32.95555, -117.09488, 100)
(24203652, '2330 Mayor`s Manor - Large Historic Home in San Diego', 'Park West', 32.72857, -117.16493999999999, 240)
(24206323, 'Family Friendly 1br House. 2 Blocks to Windansea!', 'La Jolla', 32.832029999999996, -117.27413999999999, 150)
(24207348, 'Grant Hill by AvantStay | Cozy & Chic SD Home | 5min to Balboa Park', 'Grant Hill', 32.7091, -117.13083999999999, 215)
(24222611, 'Dog Friendly Beach Cottage - 1 Block to the Water', 'Ocean Beach', 32.75246, -117.24936000000001, 350)
(24227559, 'NEW Beach Gem 2bd/2bth Loft, Parking & Laundry', 'Pacific Beach', 32.803509999999996, -11

(30012248, 'La Jolla Beach Cottage Gem', 'La Jolla', 32.836009999999995, -117.27601999999999, 132)
(30013496, 'Dover Shores 3b', 'Mission Bay', 32.76643, -117.25063, 350)
(30015491, 'South San Diego Private Home!', 'Southwest', 32.60405, -117.07396000000001, 159)
(30016553, 'LUXURY 4LEVEL TOWNHOUSE - PRIVATE ROOFDECK VIEWS!!', 'Little Italy', 32.72609, -117.16821999999999, 350)
(30017261, 'LUXURY 4LEVEL Townhouse!! AMAZING Roofdeck Views!', 'Little Italy', 32.72472, -117.16691999999999, 360)
(30022396, 'Quick, quiet, quaint', 'Webster', 32.73417, -117.07996999999999, 40)
(30038922, 'Incredible Ballpark Condo with Direct Views', 'East Village', 32.70857, -117.1542, 117)
(30042400, 'Charming SD Boho Home minutes from the Trolley!', 'Sky Line', 32.70598, -117.04915, 47)
(30042864, '☀️ Family BEACH BUNGALOW | Private Front Patio w/BBQ!', 'Mission Bay', 32.762859999999996, -117.25043000000001, 495)
(30046184, '1300 Sq. Ft. Minimalistic 2BD/1B Condo with a View', 'Midtown', 32.73815, -117.17

(36051127, 'Walk to Shelter Island, cozy studio!', 'Roseville', 32.72182, -117.23832, 92)
(36057058, 'Comic Con at a closed business for big group!', 'Gaslamp Quarter', 32.711529999999996, -117.15991000000001, 1200)
(36066101, 'Perfect Getaway! Near GasLamp, Coronado, Balboa', 'Balboa Park', 32.71712, -117.14041, 106)
(36067004, 'Quiet room on Sequoia', 'Mission Bay', 32.7903, -117.23411999999999, 50)
(36067461, 'Home w/Yard, & Parking! Ultra clean & Central 2All', 'Memorial', 32.70182, -117.13738000000001, 145)
(36070831, 'Perfect for Beach, Del Mar Track & Youth Sports', 'Carmel Valley', 32.95152, -117.23018, 275)
(36073475, 'Great Location Near Convention Center', 'Park West', 32.72695, -117.16256000000001, 99)
(36075176, 'Captivating Canyon Oasis With Fun San Diego Views', 'Midtown', 32.743970000000004, -117.17748999999999, 195)
(36076135, 'Spanish Harbor House, Sleeps 8', 'Wooded Area', 32.7203, -117.23638000000001, 350)
(36077902, 'Radiant Award Winning Furniture & Downtown Views

(40969637, '*SD Glamour Apt! BEST AREA+SANITIZED+PRIVATE+SAFE', 'Balboa Park', 32.71443, -117.1464, 65)
(40969730, 'SD Stylish Modern Interior! SANITIZED+PRIVATE+SAFE', 'Grant Hill', 32.71192, -117.14421000000002, 65)
(40969758, 'Modern Corporate East Village Apt', 'East Village', 32.714490000000005, -117.14818999999999, 129)
(40970013, '1 Bedroom in Heart of Pacific Beach', 'Pacific Beach', 32.80164, -117.24043, 150)
(40971271, 'Ocean View From Your Window in Mission Beach ', 'Mission Bay', 32.77949, -117.25192, 138)
(40971671, 'Modern San Diego Dream Home! 5BR - Sleeps 11!', 'Pacific Beach', 32.79598, -117.23635, 2999)
(40973960, 'Beautiful apartment, very well located', 'Mission Valley', 32.77301, -117.14098999999999, 249)
(40980372, 'Beach Townhouse', 'Pacific Beach', 32.802240000000005, -117.23146000000001, 105)
(40983817, 'Professionals Paradise', 'East Village', 32.71365, -117.15444, 139)
(40985370, 'Very Nice Business Stay', 'East Village', 32.71255, -117.15602, 119)
(40986517,

(44380257, 'SD Spacious Suite! 2BR~SANITIZED+PRIVATE+SAFE', 'East Village', 32.71298, -117.1486, 342)
(44380335, '**SD PENTHOUSE Masterpiece! SANITIZED+PRIVATE+SAFE', 'East Village', 32.712540000000004, -117.14966000000001, 342)
(44381264, 'SD BRAND NEW PENTHOUSE+SANITIZED+PRIVATE+SAFE', 'East Village', 32.71441, -117.14953, 342)
(44381351, 'SD SPECTACULAR VIEW! SANITIZED+PRIVATE+SAFE', 'East Village', 32.710370000000005, -117.15073000000001, 342)
(44381867, 'Steps to Sand! 2BR + Loft!! Highest Roof Deck!!!!', 'Mission Bay', 32.77384, -117.25182, 1999)
(44382084, 'Little House on Orchid Way', 'Mission Bay', 32.579840000000004, -117.06618, 50)
(44382341, 'Modernly Furnished, Dual Master w/ pool & balcony', 'Pacific Beach', 32.80447, -117.23351000000001, 225)
(44382631, 'San Diego Sleek & Stylish: SANITIZED+PRIVATE+SAFE', 'East Village', 32.71197, -117.15221000000001, 114)
(44382666, 'San Diego SKY LINE VIEW! SANITIZED+PRIVATE+SAFE', 'East Village', 32.71252, -117.15265, 342)
(44383055, 

In [14]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# # Save reference to the table
Burritos = Base.classes.burritos
Listings = Base.classes.listings
Summary = Base.classes.summary

AttributeError: summary